In [39]:
import random
from numpy import *
from functools import reduce
import pandas as pd

In [40]:
def sigmoid(inX):
    return 1.0 / (1 + exp(-inX))

In [41]:
class Node(object):
    def __init__(self,layer_index,node_index):
        self.layer_index=layer_index
        self.node_index=node_index
        self.downstream=[]
        self.upstream=[]
        self.output=0
        self.delta=0
    
    #设置节点的输出值，如果节点属于输入层会用到这个函数
    def set_output(self,output):
        self.output=output 
    
    #添加一个到下游节点的连接
    def append_downstream_connection(self,conn):
        self.downstream.append(conn)
        
    #添加一个到上游节点的连接
    def append_upstream_connection(self,conn):
        self.upstream.append(conn)
        
    #计算节点的输出
    def calc_output(self):
        output=reduce(lambda ret,conn:ret+conn.upstream_node.output*conn.weight,self.upstream,0)
        self.output=sigmoid(output)
        
    #计算隐藏层delta
    def calc_hidden_layer_delta(self):
        downstream_delta=reduce(lambda ret,conn:ret+conn.downstream_node.delta*conn.weight,self.downstream,0.0)
        self.delta=self.output*(1-self.output)*downstream_delta
        
    #计算输出层delta
    def calc_output_layer_delta(self,label):
        self.delta=self.output*(1-self.output)*(label-self.output)
        
    #打印节点信息
    def __str__(self):
        node_str = '%u-%u: output: %f delta: %f' % (self.layer_index, self.node_index, self.output, self.delta)
        downstream_str = reduce(lambda ret, conn: ret + '\n\t' + str(conn), self.downstream, '')
        upstream_str = reduce(lambda ret, conn: ret + '\n\t' + str(conn), self.upstream, '')
        return node_str + '\n\tdownstream:' + downstream_str + '\n\tupstream:' + upstream_str 


In [42]:
#实现一个输出恒为1的节点（计算偏置项Wb时需要）
class ConstNode(object):
    def __init__(self,layer_index,node_index):
        self.layer_index=layer_index
        self.node_index=node_index
        self.downstream=[]
        self.output=1
    
    #添加一个到下游节点的连接
    def append_downstream_connection(self,conn):
        self.downstream.append(conn)
        
    #计算隐藏层delta
    def calc_hidden_layer_delta(self):
        downstream_delta=reduce(lambda ret,conn:ret+conn.downstream_node.delta*conn.weight,self.downstream,0.0)
        self.delta=self.output*(1-self.output)*downstream_delta
        
    #打印节点信息
    def __str__(self):
        node_str = '%u-%u: output: 1' % (self.layer_index, self.node_index)
        downstream_str = reduce(lambda ret, conn: ret + '\n\t' + str(conn), self.downstream, '')
        return node_str + '\n\tdownstream:' + downstream_str

In [43]:
#Layer对象，负责初始化一层。此外，作为Node的集合对象，提供对Node集合的操作
class Layer(object):
    def __init__(self,layer_index,node_count):
        self.layer_index=layer_index
        self.nodes=[]
        for i in range(node_count):
            self.nodes.append(Node(layer_index,i))
        self.nodes.append(ConstNode(layer_index,node_count))
        
    #设置层的输出值，如果层是输入层会用到这个函数
    def set_output(self,data):
        for i in range(len(data)):
            self.nodes[i].set_output(data[i])
            
    #计算层的输出向量
    def calc_output(self):
        for node in self.nodes[:-1]:#不包括最后一个节点,即ConstNode
            node.calc_output()
            
    #打印层的信息
    def dump(self):
        for node in self.nodes:
            print(node)

In [44]:
#Connection对象，主要记录连接的权重，以及该连接所关联的上下游节点
class Connection(object):
    def __init__(self,upstream_node,downstream_node):
        self.upstream_node=upstream_node
        self.downstream_node=downstream_node
        self.weight=random.uniform(-0.1,0.1)
        self.gradient=0.0
        
    def calc_gradient(self):
        self.gradient=self.downstream_node.delta*self.upstream_node.output
        
    def get_gradient(self):
        return self.gradient
    
    def update_weight(self,rate):
        self.calc_gradient()
        self.weight+=rate*self.gradient
        
    def __str__(self):
        return '(%u-%u) -> (%u-%u) = %f' % (
            self.upstream_node.layer_index, 
            self.upstream_node.node_index,
            self.downstream_node.layer_index, 
            self.downstream_node.node_index, 
            self.weight)

In [45]:
#Connections对象，提供Connection集合操作
class Connections(object):
    def __init__(self):
        self.connections=[]
    
    def add_connection(self,connection):
        self.connections.append(connection)
        
    def dump(self):
        for conn in self.connections:
            print(conn)

In [46]:
#Network对象，提供API
class Network(object):
    #初始化一个全连接神经网络，layers：二维数组，描述神经网络每层节点数
    def __init__(self,layers):
        self.connections=Connections()
        self.layers=[]
        layer_count=len(layers)
        node_count=0
        for i in range(layer_count):
            self.layers.append(Layer(i,layers[i]))
        for layer in range(layer_count-1):
            connections=[Connection(upstream_node,downstream_node)
                        for upstream_node in self.layers[layer].nodes
                        for downstream_node in self.layers[layer+1].nodes[:-1]]
            for conn in connections:
                self.connections.add_connection(conn)
                conn.downstream_node.append_upstream_connection(conn)
                conn.upstream_node.append_downstream_connection(conn)
    
    #训练神经网络
    def train(self,labels,data_set,rate,iteration):
        for i in range(iteration):
            for d in range(len(data_set)):
                self.train_one_sample(labels[d],data_set[d],rate)
                # print 'sample %d training finished' % d
    
    #用一个样本训练网络
    def train_one_sample(self,label,sample,rate):
        self.predict(sample)
        self.calc_delta(label)
        self.update_weight(rate)
    
    #计算每个节点的delta
    #self.layers[-2::-1]是Python中一种切片操作，用于获取列表或其他可迭代对象的子序列。
    #-2表示从倒数第二个元素开始。在这里，它指的是self.layers列表中倒数第二个元素，即倒数第二层的节点列表。
    #::-1表示反向迭代，即从倒数第二个元素开始逆序向前遍历。这样做的目的是从倒数第二层开始，依次向前遍历神经网络的隐藏层。
    #因此，self.layers[-2::-1]返回了从倒数第二层开始到第一层（不包括第一层）的所有隐藏层的节点列表。
    def calc_delta(self,label):
        output_nodes=self.layers[-1].nodes
        for i in range(len(label)):
            output_nodes[i].calc_output_layer_delta(label[i])
        for layer in self.layers[-2::-1]:
            for node in layer.nodes:
                node.calc_hidden_layer_delta()
                
    #更新每个连接的权重
    def update_weight(self,rate):
        for layer in self.layers[:-1]:
            for node in layer.nodes:
                for conn in node.downstream:
                    conn.update_weight(rate)
                
    #计算每个连接的梯度
    def calc_gradient(self):
        for layer in self.layers[:-1]:
            for node in layer.nodes:
                for conn in node.downstream:
                    conn.calc_gradient()
                    
    #获得网络在一个样本下，每个连接上的梯度
    def get_gradient(self,label,sample):
        self.predict(sample)
        self.calc_delta(label)
        self.calc_gradient()
        
    #根据输入的样本预测输出值
    def predict(self,sample):
        self.layers[0].set_output(sample)
        for i in range(1,len(self.layers)):
            self.layers[i].calc_output()
        return list(map(lambda node:node.output,self.layers[-1].nodes[:-1]))
        #获取这一层的节点时去掉最后一个节点。不过，最后一个节点通常是用于偏置项的
    
    #打印网络信息
    def dump(self):
        for layer in self.layers:
            layer.dump()

IndentationError: expected an indented block after 'for' statement on line 15 (2153567086.py, line 16)

In [47]:
def gradient_check(network, sample_feature, sample_label):
    '''
    梯度检查
    network: 神经网络对象
    sample_feature: 样本的特征
    sample_label: 样本的标签
    '''
    # 计算网络误差
    network_error = lambda vec1, vec2: \
            0.5 * reduce(lambda a, b: a + b, 
                      list(map(lambda v: (v[0] - v[1]) * (v[0] - v[1]),
                          zip(vec1, vec2))))

    # 获取网络在当前样本下每个连接的梯度
    network.get_gradient(sample_feature, sample_label)

    # 对每个权重做梯度检查    
    for conn in network.connections.connections: 
        # 获取指定连接的梯度
        actual_gradient = conn.get_gradient()
    
        # 增加一个很小的值，计算网络的误差
        epsilon = 0.0001
        conn.weight += epsilon
        error1 = network_error(network.predict(sample_feature), sample_label)
    
        # 减去一个很小的值，计算网络的误差
        conn.weight -= 2 * epsilon # 刚才加过了一次，因此这里需要减去2倍
        error2 = network_error(network.predict(sample_feature), sample_label)
    
        # 根据式6计算期望的梯度值
        expected_gradient = (error2 - error1) / (2 * epsilon)
    
        # 打印
        print('expected gradient: \t%f\nactual gradient: \t%f' % (
            expected_gradient, actual_gradient))

def gradient_check_test():
    net = Network([2, 2, 2])
    sample_feature = [0.9, 0.1]
    sample_label = [0.9, 0.1]
    gradient_check(net, sample_feature, sample_label)        
        
''''
首先，代码创建了一个 Normalizer 的实例，这很可能是一个用于对数据进行归一化处理的类的实例。
然后，代码初始化了两个空列表 data_set 和 labels，用于存储训练数据和相应的标签。
最后，函数返回了生成的标签列表和数据集列表。
'''
def train_data_set():
    data = pd.read_csv('data/iris/iris.data', header=None)
    x, y = data[[0, 1, 2, 3]], pd.Categorical(data[4]).codes
    data_set = x.values.tolist()
    labels = _convert_label(y.tolist())
    # print(type(labels), labels)
    # print(data_set)
    return labels, data_set


# 把鸢尾花分类转化成ond-hot的三维数组
def _convert_label(lables):
    new_list = []
    for item in lables:
        if item == 0:
            new_list.append([0.9, 0.1, 0.1])
        elif item == 1:
            new_list.append([0.1, 0.9, 0.1])
        elif item == 2:
            new_list.append([0.1, 0.1, 0.9])
    return new_list


def train(network):
    # 断言语句，用来确保network是一个对象的实例。如果不是，则会引发AssertionError异常
    assert isinstance(network, object)
    labels, data_set = train_data_set()
    network.train(labels, data_set, 0.3, 100)

In [50]:
if __name__ == '__main__':
    # gradient_check_test()
    # 设置神经网络初始化参数，初始化神经网络,列表长度表示网络层数，每个数字表示每一层节点个数
    test_samples = [5.4, 3.4, 1.7, 0.2]
    net = Network([4, 2, 3])
    train(net)
    # net.dump()
    print(net.predict(test_samples))

0-0: output: 5.900000 delta: -0.000000
	downstream:
	upstream:
0-1: output: 3.000000 delta: -0.000000
	downstream:
	upstream:
0-2: output: 5.100000 delta: -0.000000
	downstream:
	upstream:
0-3: output: 1.800000 delta: -0.000000
	downstream:
	upstream:
0-4: output: 1
	downstream:
1-0: output: 0.500000 delta: 0.008565
	downstream:
	(1-0) -> (2-0) = -0.582422
	(1-0) -> (2-1) = -0.371922
	(1-0) -> (2-2) = 0.278147
	upstream:
1-1: output: 0.500000 delta: 0.006848
	downstream:
	(1-1) -> (2-0) = -0.508487
	(1-1) -> (2-1) = -0.384890
	(1-1) -> (2-2) = 0.153316
	upstream:
1-2: output: 1
	downstream:
	(1-2) -> (2-0) = -0.921614
	(1-2) -> (2-1) = -0.609817
	(1-2) -> (2-2) = 0.518770
2-0: output: 0.188315 delta: -0.013499
	downstream:
	upstream:
	(1-0) -> (2-0) = -0.582422
	(1-1) -> (2-0) = -0.508487
	(1-2) -> (2-0) = -0.921614
2-1: output: 0.274362 delta: -0.034713
	downstream:
	upstream:
	(1-0) -> (2-1) = -0.371922
	(1-1) -> (2-1) = -0.384890
	(1-2) -> (2-1) = -0.609817
2-2: output: 0.670782 del